In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import max_functions as mf

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

path = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=11.5eV' # use your path
all_files = glob.glob(path + "/*.dat")

li = []
cosphi_th = []
colnames=["phi","cos(theta)","value"] 

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None) 
    # or delim_whitespace=True, it is faste
    # r"\s+" is a regex (regular expression)
    df["filename"]=filename.split("\\")[-1].split(".")[0] # adding a column with the file name
    df["filename"]=df["filename"].str.replace(" ","") # corrects for spaces
    cosphi_th.append((int(df["filename"].str.split("_")[0][0]),int(df["filename"].str.split("_")[0][1])))
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th=np.array(cosphi_th)
frame = pd.concat(li, axis=0)
frame_srt=frame.groupby("filename")

In [ ]:
frame.loc[frame["filename"] == "1_1"] # first way of selecting the right file
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
# Single 2D maps from pandas without tranforming into numpy
frame.loc[frame["filename"] == "1_1"].plot.hexbin(x="phi", y="cos(theta)", C="value", sharex=False, gridsize=50)

In [ ]:
# converting to numpy arrays to plot
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)
counts = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # values

print(phi.shape, ctheta.shape, counts.shape)
print(type(phi), type(ctheta), type(counts))
counts_matrix = counts.reshape(200,100) # reshape(rows=x,columns=y)

print("number of ROWS (x) = ", len(counts_matrix[:,1]), "number of COLUMNS (y for reshaping) = ", len(counts_matrix[1,:]))

In [ ]:
# plt.imshow(z_matrix)
plt.imshow(counts_matrix.T,extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], aspect=180)
ax = plt.axes()
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
plt.colorbar()

In [ ]:
# countour plot
cmap = mpl.cm.viridis
cmap.set_under('w')


fig1, ax = plt.subplots(constrained_layout=True)

CS = ax.contourf(phi,ctheta,counts_matrix.T, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
fig1.colorbar(CS)

# OLD WAY OF PROCEDING!! Note unique values and extent, all the 72 plots
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)

fig = plt.figure(figsize=(50, 24),dpi=80) # figsize in inche by defauls: 1 in = 2.8 cm
for filename,group,i in zip(frame_srt,frame_srt,range(1,73)) :
    counts_str = group[1]["value"].to_numpy()
    counts_matrix = counts_str.reshape(200,100)
    ax = fig.add_subplot(6, 12, i)
    ax.contourf(phi,ctheta,counts_matrix, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)
    ax.text(i, 1+i//12, i, ha="center", va="center", color="red")

fig.tight_layout()
# plt.show()

#OLD WAY OF PROCEEDING

fig = plt.figure(figsize=(25, 12),dpi=40) # downsized
for filename,group,i in zip(frame_srt,frame_srt,range(72)):
    counts_str = group[1]["value"].to_numpy()
    ax = fig.add_subplot(6, 12, i+1)
    ax.contourf(phi.reshape(200,100),ctheta.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
    ax.text(i+1, i//12+1, cosphi_th[i], ha="center", va="center", color="red")

fig.tight_layout()

In [ ]:
cosphi_th;

In [ ]:
# all the 72 plots
# the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
# 6 levels of cos(theta) (y), 12 levels of phi (x)
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

for filename,group,i,ax in zip(frame_srt,frame_srt,range(72),axes.flatten()) :
    counts_str = group[1]["value"].to_numpy()
    ax.contourf(phi.reshape(200,100),ctheta.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
    ax.text(0, 0, cosphi_th[i], ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()


# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :

# OLD WAY OF PROCEDING!! Note unique values and extent, all the 72 plots
    phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
    ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)

    fig = plt.figure(figsize=(50, 24),dpi=80) # figsize in inche by defauls: 1 in = 2.8 cm
    for filename,group,i in zip(frame_srt,frame_srt,range(1,73)) :
        counts_str = group[1]["value"].to_numpy()
        counts_matrix = counts_str.reshape(200,100)
        ax = fig.add_subplot(6, 12, i)
        ax.contourf(phi,ctheta,counts_matrix, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)
        ax.text(i, 1+i//12, i, ha="center", va="center", color="red")
    
    fig.tight_layout()
    # plt.show()"

  #OLD WAY OF PROCEEDING
    fig = plt.figure(figsize=(25, 12),dpi=40) # downsized
    for filename,group,i in zip(frame_srt,frame_srt,range(72)):
        counts_str = group[1]["value"].to_numpy()
        ax = fig.add_subplot(6, 12, i+1)
        ax.contourf(phi.reshape(200,100),ctheta.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
        ax.text(i+1, i//12+1, cosphi_th[i], ha="center", va="center", color="red")
    fig.tight_layout()

In [ ]:
# all the 72 plots: no borders & sharing axis


phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)

# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row') figsize in inche by defauls: 1 in = 2.8 cm
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row') #downsized
fig.subplots_adjust(hspace = .5, wspace=.5)

for filename,group,ax in zip(frame_srt,frame_srt,axes.flatten()) :
    counts_str = group[1]["value"].to_numpy()
    counts_matrix = counts_str.reshape(200,100)
    ax.contourf(phi.reshape(200,100),ctheta.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
    
fig.tight_layout()
plt.show()

In [ ]:
from numpy.core.defchararray import multiply


counts_matrixS=[]
counts_matrixR=[]
cos_projthS=[]
cos_projthR=[]
phi_projthS=[]
phi_projthR=[]

for filename,group,i in zip(frame_srt,frame_srt,range(72)) :
# for filename,group,i in zip(frame_srt,frame_srt,range(72)) :
    # print(filename)
    counts_str = group[1]["value"].to_numpy()
    counts_matrixS.append(counts_str.reshape(200,100))
    cos_projthS.append(counts_matrixS[i].sum(axis=0))
    phi_projthS.append(counts_matrixS[i].sum(axis=1))
    tempcos=np.flip(counts_matrixS[i], axis=0)
    tempphi=np.flip(tempcos, axis=1)
    counts_matrixR.append(tempphi)
    cos_projthR.append(counts_matrixR[i].sum(axis=0))
    phi_projthR.append(counts_matrixR[i].sum(axis=1))

cos_projthS=np.array(cos_projthS)
cos_projthR=np.array(cos_projthR)
phi_projthS=np.array(phi_projthS)
phi_projthR=np.array(phi_projthR)

np.array(counts_matrix).shape
print(np.array(cos_projthS).shape)
print(np.array(cos_projthR).shape)

In [ ]:
cPECD =  np.divide(np.subtract(cos_projthR,cos_projthS),np.add(cos_projthR,cos_projthS))
errPECD = np.sqrt(1/(cos_projthR+cos_projthS))

In [ ]:
plt.plot(cos_projthR[0])
plt.plot(cos_projthS[0])

In [ ]:
ctheta_lin=np.unique(ctheta)
# print(ctheta_lin, ctheta_lin.shape)

In [ ]:
# modified with error
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

class LeastSquares:
    """
    Generic least-squares cost function with error.
    """
    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)
        self.err = np.array(err)
        # self.err = err

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = np.sum((self.y - ym)**2/(self.err**2))
        return chi2

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

def PECD6(x, b1, b2, b3, b4, b5, b6):
    return ((b1*x + 0.5*b3*(5*x**3-3*x) +0.125*b5*(63*x**5-70*x**3+15*x)) / (1 + 0.5*b2*(3*x**2-1) + 0.125*b4*(35*x**4-30**2+3) + 0.0625*b6*(231*x**6-315*x**4+105*x**2-5)))

# param_matrix=np.zeros((6,72)) #nparray initialization
param_matrix = [[0 for i in range(72)] for j in range(6)] #i col, j row

for i in range(72): #fixe because of the photon directions, equal to range(len(cosphi))
    #print(i)
    x_data = ctheta_lin
    y_data = cPECD[i]
    y_err = errPECD[i]

    lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

    m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
    # m = Minuit(lsq, error_b1=0.01, error_b2=0.01, limit=((-1, 1), (-1, 1)), name=("b1", "b2"), pedantic=False)
    m.migrad() # run optimiser
    m.hesse() # run covariance estimator
    for j,p in zip(range(len(m.parameters)),m.parameters): # len = 6
        param_matrix[j][i] = ((m.values[p], m.errors[p]))

    # for p in m.parameters:
        # print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))

In [ ]:
import matplotlib as mpl
from matplotlib import cm

#the following converts a cmap into plotly language 
magma_cmap = mpl.cm.get_cmap('magma')
seismic_cmap = mpl.cm.get_cmap('seismic')

magma_rgb = []
seismic_rgb = []
norm = mpl.colors.Normalize(vmin=0, vmax=255)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(magma_cmap(norm(i)))
       magma_rgb.append(k)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(seismic_cmap(norm(i)))
       seismic_rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

Magma_r = matplotlib_to_plotly(magma_cmap, 255)
Seismic_r = matplotlib_to_plotly(seismic_cmap, 255)

In [ ]:
param_matrix=np.array(param_matrix) #numpy tranformation!!!
print(param_matrix.shape)
param_matrix[0,:,0].shape

In [ ]:
plt.plot(param_matrix[0,:,0])

In [ ]:
from numpy.lib.function_base import meshgrid
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools


param_matrix=np.array(param_matrix) #numpy tranformation!!!
phicos_MFPAD = np.array(list(itertools.product(np.linspace(-180,180,12).tolist(), np.linspace(-1,1,6).tolist())))

go_phi=[col[0] for col in phicos_MFPAD]
go_cos=[col[1] for col in phicos_MFPAD]

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r), 1, 1)
fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0.75, colorscale=Seismic_r), 1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=900,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()

In [ ]:
#NOTE: THE SCALE IS FAKE FOR PLOT 2 AND 6!

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()

fig = make_subplots(rows=6, cols=2, vertical_spacing=0.015,
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

for i in range(len(param_matrix)):
    if i==1 or i==5:
        fig.add_trace(go.Contour(z=param_matrix[i,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix[i,:,0], x=go_phi, y=go_cos, line_smoothing=0.85, colorscale=Seismic_r, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 2)
    else:
        fig.add_trace(go.Contour(z=param_matrix[i,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, legendgroup="yes", contours=dict(start=-0.2, end=0.2, size=0.04)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix[i,:,0], x=go_phi, y=go_cos, line_smoothing=0.85, colorscale=Seismic_r, legendgroup="yes", contours=dict(start=-0.2, end=0.2, size=0.04)), i+1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    # width=800,
    height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()